In [118]:
import glob
import math
import pandas as pd
import re
import urllib.request
import zipfile
years = range(2008, 2016)
path = '/data/parking/'

In [9]:
def download_data(years, path):
    for year in years:
        print('downloading %d' % year)
        urllib.request.urlretrieve('http://opendata.toronto.ca/revenue/parking/ticket/parking_tickets_%d.zip' % year,
                                   '%s/%d.zip' % (path, year))
        
download_data(years, path)

downloading 2008
downloading 2009
downloading 2010
downloading 2011
downloading 2012
downloading 2013
downloading 2014
downloading 2015


In [12]:
def unzip_data(years, path):
    for year in years:
        with zipfile.ZipFile('%s/%d.zip' % (path, year)) as zf:
            zf.extractall(path)
        
unzip_data(years, path)

In [37]:
def read_all_data(path, cols):
    df = None
    for fname in glob.glob('%s/*.csv' % path):
        print(fname)
        df2 = pd.read_csv(fname,
                          encoding='cp1252',  # unfortunately, not utf8
                          quoting=csv.QUOTE_NONE,
                          usecols=cols)
        if df is None:
            df = df2
        else:
            df = pd.concat((df, df2))
    return df
                             
location_df = read_all_data(path, ['location2'])

/data/parking/Parking_Tags_data_2008.csv
/data/parking/Parking_Tags_data_2009.csv
/data/parking/Parking_Tags_data_2010.csv
/data/parking/Parking_Tags_data_2011.csv
/data/parking/Parking_Tags_Data_2012.csv
/data/parking/Parking_Tags_Data_2013.csv
/data/parking/Parking_Tags_Data_2014_1.csv
/data/parking/Parking_Tags_Data_2014_2.csv
/data/parking/Parking_Tags_Data_2014_3.csv
/data/parking/Parking_Tags_Data_2014_4.csv
/data/parking/Parking_Tags_Data_2015_1.csv
/data/parking/Parking_Tags_Data_2015_2.csv
/data/parking/Parking_Tags_Data_2015_3.csv


In [56]:
locations = [x for x in location_df['location2'] if x and type(x) == str]
# total location strings.
len(locations)

15588133

In [59]:
# unique addresses.
locations_u = set(locations)
len(locations_u)

509625

In [60]:
# unique street names.
len(set(re.sub(r'[0-9]', '', x) for x in locations_u))

47046

In [117]:
def do_round(numbers, n):
    # n=number of sig digits.
    v = 10**n
    return str(math.ceil(int(numbers) / v) * v)
    
def collapse_digits(address, n):
    return re.sub(r'([0-9]+)',
           lambda x: do_round(x.groups()[0], n),
           address)

test_address = '2743 VICTORIA PARK AVE'
print(collapse_digits(test_address, 4))
print(collapse_digits(test_address, 3))
print(collapse_digits(test_address, 2))
print(collapse_digits(test_address, 1))
test_address = '123 MAIN ST'
print(collapse_digits(test_address, 4))
print(collapse_digits(test_address, 3))
print(collapse_digits(test_address, 2))
print(collapse_digits(test_address, 1))

10000 VICTORIA PARK AVE
3000 VICTORIA PARK AVE
2800 VICTORIA PARK AVE
2750 VICTORIA PARK AVE
10000 MAIN ST
1000 MAIN ST
200 MAIN ST
130 MAIN ST


In [116]:
# How does collapsing addresses reduce size?
for i in [4,3,2,1]:
    print('rounding to %d yields %d addresses' %
          (10**i, len(set(collapse_digits(x, i) for x in locations_u))))

collapsing to 10000 yields 48451 addresses
collapsing to 1000 yields 52594 addresses
collapsing to 100 yields 72369 addresses
collapsing to 10 yields 168696 addresses
